In [1]:
#sources:
#https://www.kaggle.com/datasets/dhruvildave/billboard-the-hot-100-songs
#https://www.kaggle.com/datasets/lukaszamora/history-of-rock-19502020?resource=download

In [2]:
from ipywidgets import interact
import ipywidgets as widgets
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets

In [3]:
class ToMuchChosenException(Exception):
    def __init__(self):
        self.message = ("You can choose only one or two attributes")
    def __str__(self):
        return self.message

In [4]:
rock = pd.read_csv('../Downloads/history-of-rock-spotify.csv')
charts = pd.read_csv('../Downloads/charts.csv')

In [5]:
charts['year'] = charts['date'].str[0:4].apply(int)
charts = charts.drop(['last-week'], axis=1)
charts = charts[charts['rank']==charts['peak-rank']]
charts['song'] = charts['song'].apply(lambda x: re.sub('\(([\w]*)( )*From ([\w" ]*)\)', "", str(x)))

In [6]:
rock = rock.drop(['index'], axis=1)
rock['name'] = rock['name'].apply(lambda x: re.sub(" - ([\w]*)( )*Remaster([\w]*)( )*([\w]*)", "", str(x)))
rock['name'] = rock['name'].apply(lambda x: re.sub(" - ([\w]*)( )*Version", "", str(x)))

In [7]:
rock_in_chart = pd.merge(rock,charts, how='inner', left_on=['name', 'artist'], right_on=['song','artist'])
rock_in_chart = rock_in_chart.sort_values('date')
rock_in_chart = rock_in_chart.drop(columns = ['time_signature', 'rank', 'danceability.1', 'key', 'release_date', 'popularity'])

In [8]:
potential_columns=['peak-rank',
                   'weeks-on-board',
                   'length',
                   'tempo',
                   'instrumentalness',
                   'acousticness',
                   'danceability',
                   'energy',
                   'liveness',
                   'loudness',
                   'speechiness',
                   'valence',
                   ]

In [9]:
years = rock_in_chart.groupby('year').count()>30
rock_in_chart = rock_in_chart.groupby('year').median()[years]

In [11]:
@interact(col_names=widgets.SelectMultiple(
    options=potential_columns,
    value=(potential_columns[0],),
    description='Columns'))
def plot_data(col_names):
    plt.figure(figsize=[12,8])
    plt.gcf().suptitle(str(col_names))
    plt.rcParams.update({'font.size': 14})
    plt.xlabel('year')
    if len(col_names) == 2:
        plt.plot(rock_in_chart[list(col_names)[0]],'b')
        plt.ylabel(col_names[0])
        ax=plt.twinx()
        ax.plot(rock_in_chart[list(col_names)[1]],'r')
        plt.ylabel(col_names[1])
    elif len(col_names) == 3:
        raise ToMuchChosenException()
    else:
        plt.ylabel(col_names[0])
        plt.plot(rock_in_chart[list(col_names)], 'b')

interactive(children=(SelectMultiple(description='Columns', index=(0,), options=('peak-rank', 'weeks-on-board'…